# Model Deployment

## Setup

In [4]:
!pip install -q huggingface_hub
!pip install -q onnx
!pip install -q onnx-tf
!pip install -q tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


ERROR: Could not find a version that satisfies the requirement tensorflow-addons (from versions: none)
ERROR: No matching distribution found for tensorflow-addons


In [54]:
# login to HF
from huggingface_hub import notebook_login
notebook_login()

## Load model and run inference

In [8]:
import v8_bigram_scaled as v8

model_path = 'models/lafontaine_gpt_v8_241011_1307.pth'
model = v8.load_model(model_path)

text = v8.run_inference(model)
print(text)


Model loaded from models/lafontaine_gpt_v8_241011_1307.pth.

Pallus qu'en des sacrifices de Maximères.
Le Dieu les escadrons pour les milles est venus,
On se font pris amuser tous les jours déplaire.
Philémon les pauvret Baudets
Qu'ils n'approchaient leur faire :
Car cultait de boire ; ils s'enfuirent gagner.
Je vous plains, je te promis, j'en serai par plus monarque des hommes,
De femme, ayent de humains, qu'un Maître de l'air,
Du restentier et des morts sa main querelle.
Le sage qui les mit sont d'en prince le croire ;
Et, s'il est aurait venir
De ce qu


## Convert model to onnx (deprecated)

In [9]:
import v8_bigram_scaled as v8

model_path = 'models/lafontaine_gpt_v8_241011_1307.pth'
model = v8.load_model(model_path)

onnx_model_path = model_path.replace('.pth', '.onnx')
v8.export_onnx_model(model, onnx_model_path)

Model loaded from models/lafontaine_gpt_v8_241011_1307.pth.
Model exported to models/lafontaine_gpt_v8_241011_1307.onnx.


## Run inference in Gradio

In [13]:
import gradio as gr
import torch
from v8_bigram_scaled import BigramLanguageModel, encode, decode

# Assuming 'BigramLanguageModel' and 'decode' are defined as in your code

class GradioInterface:
    def __init__(self, model_path=None):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model = self.load_model(model_path)
        self.model.eval()

    def load_model(self, model_path):
        model = BigramLanguageModel().to(self.device)
        if model_path:
            model.load_state_dict(torch.load(model_path, map_location=self.device))
        return model

    def generate_text(self, input_text, max_tokens=100):
        context = torch.tensor([encode(input_text)], dtype=torch.long, device=self.device)
        output = self.model.generate(context, max_new_tokens=max_tokens)
        return decode(output[0].tolist())

# Load the model
model_path = "models/lafontaine_gpt_v8_241011_1307.pth"
model_interface = GradioInterface(model_path)

# Define Gradio interface
gr_interface = gr.Interface(
    fn=model_interface.generate_text,
    inputs=["text", gr.Slider(50, 500)],
    outputs="text",
    description="Bigram Language Model text generation. Enter some text, and the model will continue it.",
    examples=[["Once upon a time"]]
)

# Launch the interface
gr_interface.launch()


C:\Users\Nath\AppData\Local\Temp\ipykernel_18284\3579305608.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=se

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## Push model to hub

In [16]:
import os
from huggingface_hub import Repository

# clone the repository
repo = Repository(local_dir="deployment/v1", clone_from="https://huggingface.co/alexdjulin/lafontaine-gpt", use_auth_token=os.environ["HF_TOKEN"])


Cloning https://huggingface.co/alexdjulin/lafontaine-gpt into local empty directory.


Download file model.safetensors:   0%|          | 25.5k/42.2M [00:00<?, ?B/s]

In [ ]:
import os
from huggingface_hub import Repository
from dotenv import load_dotenv
load_dotenv()

repo = Repository(
    local_dir="deployment/v1",
    clone_from="https://huggingface.co/alexdjulin/lafontaine-gpt",
    use_auth_token=os.environ.get("HF_TOKEN")
)
repo.push_to_hub()
